### Kaggle Predicting Red Hat Business Value

As this my first kernel, I would very much appreciate some feedback.  While i'm confident that most of my process, particularly the initial clearning and transformation from cateogrical to numeric are correct, i'm more unsure about the steps I took in re: to train, test, split, etc.  

Thank you to everyone who contributes with tips and kernels of their own!

In [56]:
# importing libraries

import numpy as np
import pandas as pd
from IPython.display import display, HTML

from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [31]:
# reading in data

people = pd.read_csv('people.csv')
activity_train = pd.read_csv('act_train.csv')
activity_test = pd.read_csv('act_test.csv')

In [32]:
# merging the dataframes into train, test

df = activity_train.merge(people, how='left', on='people_id' )
df_test = activity_test.merge(people, how='left', on='people_id' )

In [33]:
# the shape of the dataframes

print df.shape
print df_test.shape

(2197291, 55)
(498687, 54)


In [34]:
# filling NaN values first

df = df.fillna('0', axis=0)
df_test = df_test.fillna('0', axis=0)

In [35]:
# taking a look at the first few rows

df.head()

,people_id,activity_id,date_x,activity_category,char_1_x,char_2_x,char_3_x,char_4_x,char_5_x,char_6_x,...,char_29,char_30,char_31,char_32,char_33,char_34,char_35,char_36,char_37,char_38
0,ppl_100,act2_1734928,2023-08-26,type 4,0,0,0,0,0,0,...,False,True,True,False,False,True,True,True,False,36
1,ppl_100,act2_2434093,2022-09-27,type 2,0,0,0,0,0,0,...,False,True,True,False,False,True,True,True,False,36
2,ppl_100,act2_3404049,2022-09-27,type 2,0,0,0,0,0,0,...,False,True,True,False,False,True,True,True,False,36
3,ppl_100,act2_3651215,2023-08-04,type 2,0,0,0,0,0,0,...,False,True,True,False,False,True,True,True,False,36
4,ppl_100,act2_4109017,2023-08-26,type 2,0,0,0,0,0,0,...,False,True,True,False,False,True,True,True,False,36


In [ ]:
df_test.head()

### preprocessing

In [36]:
# a multi-column LabelEncoder()

# this solution for applying LabelEncoder() across multiple columns was suggested in the following thread
# http://stackoverflow.com/questions/24458645/label-encoding-across-multiple-columns-in-scikit-learn

# I like this solution but is it the most efficient?  Would another method be more practical, particularly if 
# applied to different type of model 

class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns 

    def fit(self,X,y=None):
        return self

    def transform(self,X):
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [37]:
# defining a processor 

def processor(data):
    data = MultiColumnLabelEncoder(columns = ['people_id','activity_id', 'activity_category', 'date_x', 'char_1_x', 'char_2_x',
                                        'char_3_x', 'char_4_x', 'char_5_x', 'char_6_x', 'char_7_x', 'char_8_x', 'char_9_x',
                                        'char_10_x', 'char_1_y', 'group_1', 'char_2_y', 'date_y', 'char_3_y', 'char_4_y',
                                        'char_5_y', 'char_6_y', 'char_7_y', 'char_8_y', 'char_9_y']).fit_transform(df)
    
    bool_map = {True:1, False:0}

    data = data.applymap(lambda x: bool_map.get(x,x))
    
    return data

In [38]:
# applying processor to training data

df_encoded = processor(df)

In [42]:
df_encoded.head()

,people_id,activity_id,date_x,activity_category,char_1_x,char_2_x,char_3_x,char_4_x,char_5_x,char_6_x,...,char_29,char_30,char_31,char_32,char_33,char_34,char_35,char_36,char_37,char_38
0,0,503691,405,3,0,0,0,0,0,0,...,0,1,1,0,0,1,1,1,0,36
1,0,832759,72,1,0,0,0,0,0,0,...,0,1,1,0,0,1,1,1,0,36
2,0,1289703,72,1,0,0,0,0,0,0,...,0,1,1,0,0,1,1,1,0,36
3,0,1406406,383,1,0,0,0,0,0,0,...,0,1,1,0,0,1,1,1,0,36
4,0,1623050,405,1,0,0,0,0,0,0,...,0,1,1,0,0,1,1,1,0,36


In [43]:
df_encoded.dtypes

people_id            int64
activity_id          int64
date_x               int64
activity_category    int64
char_1_x             int64
char_2_x             int64
char_3_x             int64
char_4_x             int64
char_5_x             int64
char_6_x             int64
char_7_x             int64
char_8_x             int64
char_9_x             int64
char_10_x            int64
outcome              int64
char_1_y             int64
group_1              int64
char_2_y             int64
date_y               int64
char_3_y             int64
char_4_y             int64
char_5_y             int64
char_6_y             int64
char_7_y             int64
char_8_y             int64
char_9_y             int64
char_10_y            int64
char_11              int64
char_12              int64
char_13              int64
char_14              int64
char_15              int64
char_16              int64
char_17              int64
char_18              int64
char_19              int64
char_20              int64
c

In [44]:
# saving to csv

# df.to_csv('train_e.csv')

In [45]:
# applying processor to test data

df_test_encoded = processor(df_test)

In [46]:
df_test_encoded.head()

,people_id,activity_id,date_x,activity_category,char_1_x,char_2_x,char_3_x,char_4_x,char_5_x,char_6_x,...,char_29,char_30,char_31,char_32,char_33,char_34,char_35,char_36,char_37,char_38
0,0,503691,405,3,0,0,0,0,0,0,...,0,1,1,0,0,1,1,1,0,36
1,0,832759,72,1,0,0,0,0,0,0,...,0,1,1,0,0,1,1,1,0,36
2,0,1289703,72,1,0,0,0,0,0,0,...,0,1,1,0,0,1,1,1,0,36
3,0,1406406,383,1,0,0,0,0,0,0,...,0,1,1,0,0,1,1,1,0,36
4,0,1623050,405,1,0,0,0,0,0,0,...,0,1,1,0,0,1,1,1,0,36


In [47]:
df_test_encoded.dtypes

people_id            int64
activity_id          int64
date_x               int64
activity_category    int64
char_1_x             int64
char_2_x             int64
char_3_x             int64
char_4_x             int64
char_5_x             int64
char_6_x             int64
char_7_x             int64
char_8_x             int64
char_9_x             int64
char_10_x            int64
outcome              int64
char_1_y             int64
group_1              int64
char_2_y             int64
date_y               int64
char_3_y             int64
char_4_y             int64
char_5_y             int64
char_6_y             int64
char_7_y             int64
char_8_y             int64
char_9_y             int64
char_10_y            int64
char_11              int64
char_12              int64
char_13              int64
char_14              int64
char_15              int64
char_16              int64
char_17              int64
char_18              int64
char_19              int64
char_20              int64
c

In [48]:
# df_test_encoded.to_csv('test_e.csv')

## modeling



In [50]:
# a look at the changes made

df.head()

,people_id,activity_id,date_x,activity_category,char_1_x,char_2_x,char_3_x,char_4_x,char_5_x,char_6_x,...,char_29,char_30,char_31,char_32,char_33,char_34,char_35,char_36,char_37,char_38
0,ppl_100,act2_1734928,2023-08-26,type 4,0,0,0,0,0,0,...,False,True,True,False,False,True,True,True,False,36
1,ppl_100,act2_2434093,2022-09-27,type 2,0,0,0,0,0,0,...,False,True,True,False,False,True,True,True,False,36
2,ppl_100,act2_3404049,2022-09-27,type 2,0,0,0,0,0,0,...,False,True,True,False,False,True,True,True,False,36
3,ppl_100,act2_3651215,2023-08-04,type 2,0,0,0,0,0,0,...,False,True,True,False,False,True,True,True,False,36
4,ppl_100,act2_4109017,2023-08-26,type 2,0,0,0,0,0,0,...,False,True,True,False,False,True,True,True,False,36


In [51]:
df_encoded.head()

,people_id,activity_id,date_x,activity_category,char_1_x,char_2_x,char_3_x,char_4_x,char_5_x,char_6_x,...,char_29,char_30,char_31,char_32,char_33,char_34,char_35,char_36,char_37,char_38
0,0,503691,405,3,0,0,0,0,0,0,...,0,1,1,0,0,1,1,1,0,36
1,0,832759,72,1,0,0,0,0,0,0,...,0,1,1,0,0,1,1,1,0,36
2,0,1289703,72,1,0,0,0,0,0,0,...,0,1,1,0,0,1,1,1,0,36
3,0,1406406,383,1,0,0,0,0,0,0,...,0,1,1,0,0,1,1,1,0,36
4,0,1623050,405,1,0,0,0,0,0,0,...,0,1,1,0,0,1,1,1,0,36


In [52]:
# defining X and y (features and target label)

X = df_encoded
y = X.pop('outcome')

In [55]:
# shape of X and y

print X.shape
print y.shape

(2197291, 54)
(2197291L,)


In [57]:
'''

train, test, split the data.  hold out 25% for test

generally if not provided a test set, this would be the way to move forward.
yet I feel something is off in my process and would love feedback!

'''

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=7)

In [58]:
# random forest classifier

model = RandomForestClassifier(77, n_jobs=-1, random_state=7)

In [59]:
# fitting the model to training data

model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=77, n_jobs=-1,
            oob_score=False, random_state=7, verbose=0, warm_start=False)

In [60]:
# scoring the model against test data

print "model score ", model.score(X_test, y_test)

model score  0.992874865971


In [61]:
# predicting test data

pred = model.predict(X_test)

In [72]:
pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [74]:
df_test_encoded.head()

,people_id,activity_id,date_x,activity_category,char_1_x,char_2_x,char_3_x,char_4_x,char_5_x,char_6_x,...,char_29,char_30,char_31,char_32,char_33,char_34,char_35,char_36,char_37,char_38
0,0,503691,405,3,0,0,0,0,0,0,...,0,1,1,0,0,1,1,1,0,36
1,0,832759,72,1,0,0,0,0,0,0,...,0,1,1,0,0,1,1,1,0,36
2,0,1289703,72,1,0,0,0,0,0,0,...,0,1,1,0,0,1,1,1,0,36
3,0,1406406,383,1,0,0,0,0,0,0,...,0,1,1,0,0,1,1,1,0,36
4,0,1623050,405,1,0,0,0,0,0,0,...,0,1,1,0,0,1,1,1,0,36


#### final thoughts

While the X_test a split of our traning data, as achieved through train, test, split, it is not the same as the actual test set provided in the data by kaggle.  For submission, this would be an issue.  For me, this competition is unique in although fairly straightforward, I haven't had to label encode categoricals on a provided test dataset.  Generally these are untouched until model prediction time.

Hopefully some other less experienced users such as myself can make some use of my inital sets in this process.